In [133]:
import pandas as pd
import os

In [134]:
# 각 시도 내 업소 데이터를 불러와서 dataset라는 dictionary에 key는 시도명으로 value는 각 시도의 업소데이터프레임을 저장한다.
file_list = os.listdir('./data/sido')

dataset = {} 
for file in file_list:  
    file_key = 'df_' + str(file.split('.')[0])
    filename = f'./data/sido/{file}'
    dataset[file_key] = pd.read_csv(filename)

C:\Users\cm\AppData\Local\Temp\ipykernel_11508\2306794549.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset[file_key] = pd.read_csv(filename)
C:\Users\cm\AppData\Local\Temp\ipykernel_11508\2306794549.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset[file_key] = pd.read_csv(filename)
C:\Users\cm\AppData\Local\Temp\ipykernel_11508\2306794549.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset[file_key] = pd.read_csv(filename)
C:\Users\cm\AppData\Local\Temp\ipykernel_11508\2306794549.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset[file_key] = pd.read_csv(filename)
C:\Users\cm\AppData\Local\Temp\ipykernel_11508\2306794549.py:8: DtypeWarning: Columns (35) have mixed types. Specify dtype option on import or set low_memory=False.

In [135]:
#저장한 데이터셋에서 외식창업관련 데이터를 확인하기 위해서 음식관련 데이터만 추출한다.
food_service = {}
for data in dataset:
    food_service[data] = dataset[data].query("상권업종대분류코드.str.contains('Q')")  
    food_service[data] = food_service[data].reset_index(drop=True)

In [136]:
#시도별로 나눠져있는 데이터들을 하나로 합친다.
df_food_service = pd.concat(food_service, ignore_index=True)

In [5]:
# 상권업종중분류코드와 분류명을 df로 만든다.
df_mid_code = pd.DataFrame(
    {'code':df_food_service["상권업종중분류코드"].value_counts().keys(),
    'name': df_food_service["상권업종중분류명"].value_counts().keys()}
    )

In [137]:
# 시도코드와 시도명을 df로 만든다.
sido_lat = [37.567167,37.540705,35.259787,35.198362,36.248647,36.557229,37.555837,34.819400,37.469221,35.798838,35.716705,36.628503,36.321655,35.126033,33.364805,35.519301,36.504073]
sido_lon = [127.190292,126.956764,128.664734,129.053922,128.664734,126.779757,128.209315,126.893113,126.573234,128.583052,127.144185,127.929344,127.378953,126.831302,126.542671,129.239078,127.249485]
df_sido_code = pd.DataFrame(
    {'code':df_food_service["시도코드"].value_counts().keys(),
    'name': df_food_service["시도명"].value_counts().keys(),
    'lat': sido_lat,
    'lon': sido_lon}
    )

In [138]:
# 시군구명이 중복되는 값들이 있어서 중복되지 않기 위해 시군구코드와 시군구명의 수를 확인해서 df를 만든다.
list_sgg_code = list(df_food_service["시군구코드"].value_counts().keys())

list_sgg_name = [df_food_service["시군구명"][df_food_service['시군구코드'] == i].value_counts().keys()[0] for i in list_sgg_code]

df_sgg_code = pd.DataFrame(
    {'code':list_sgg_code,
    'name': list_sgg_name}
    )

In [8]:
# 행정동명도 중복되는 값들이 있어서 시군구코드와 같은 방법을 이용해서 행정동코드와 행정동명의 df를 만든다.
list_hjd_code = list(df_food_service["행정동코드"].value_counts().keys())

list_hjd_name = [df_food_service["행정동명"][df_food_service['행정동코드'] == i].value_counts().keys()[0] for i in list_hjd_code]

df_hjd_code = pd.DataFrame(
    {'code':list_hjd_code,
    'name': list_hjd_name}
    )

In [9]:
# 상가업소번호, 업소명, 도로명주소, 위도, 경도의 df를 만든다.
df_shop = pd.DataFrame({
    "code" : df_food_service["상가업소번호"],
    "name" : df_food_service["상호명"],
    "address" : df_food_service["도로명주소"],
    "lat" : df_food_service["위도"],
    "lon" : df_food_service["경도"]
})

In [10]:
# 모든 업소의 한글 제외한 코드만으로 이루어진 df를 만든다.
df = df_food_service.iloc[:,[0,5,11,13,15]]
df = df.reset_index(drop=True)

In [11]:
# 시도별 총 음식점 숫자를 보여주는 df을 만든다.
sido = {}
for i in df_sido_code["code"]:
    sido[df_sido_code.loc[df_sido_code["code"]==i]["name"].values[0]] = len(df.query(f"시도코드 == {i}"))

df_sido_num = pd.DataFrame({
    "name" : list(sido.keys()),
    "num" : list(sido.values()),
})

In [12]:
# 용량을 줄이기위해 "상권업종중분류코드","시도코드", '시군구코드', '행정동코드'의 데이터타입을 카테고리로 변환시킨다.
for i in ["상권업종중분류코드","시도코드", '시군구코드', '행정동코드']:
    df = df.astype({i:'category'})

In [13]:
# 만든 df들을 csv파일로 만든다.
df_list = {'df': df,
'df_mid_code': df_mid_code,
'df_sido_code': df_sido_code,
'df_sgg_code': df_sgg_code,
'df_hjd_code': df_hjd_code,
'df_shop': df_shop,
'df_sido_num':df_sido_num
}

for i in df_list:
    df_list[i].to_csv(f'./data/{i}.csv', 
              sep=",", 
              na_rep="NaN"
              )

In [5]:
df_sido_code = pd.read_csv('./data/df_sido_code.csv', index_col=0)
df_sido_num = pd.read_csv('./data/df_sido_num.csv', index_col=0)
df = pd.read_csv('./data/df.csv', index_col=0)
df_mid_code = pd.read_csv('./data/df_mid_code.csv', index_col=0)
df_shop = pd.read_csv('./data/df_shop.csv', index_col=0)